<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_GRAPE%203%20Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator - 3 Level System


In [4]:
!pip install ARC-Alkali-Rydberg-Calculator
!pip install qutip

     |████████████████████████████████| 3.5MB 315kB/s 
  Created wheel for qutip: filename=qutip-4.5.0-cp36-cp36m-linux_x86_64.whl size=12882883 sha256=c602e64d337ecf83063674cc37b1de3100a9f3b389d31b5facc48925bab42b0c
  Stored in directory: /root/.cache/pip/wheels/c9/85/e8/3fbad9a0816141b4c5f1d73d8a880ed91265fea84192cbe37b
Successfully built qutip


In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
from arc import *
from qutip import *
import matplotlib.pyplot as plt


print(tf.__version__)

2.1.0


In [0]:
######Set all systm parameters: (using ARC)
GHz = 1E9
MHz = 1E6 
kHz = 1E3

ms = 1E-3
us = 1E-6
ns = 1E-9

atom=Rubidium()

##Intermediate excited state: 6P3/2
n_i = 6
l_i = 1
j_i = 1.5
T_i = atom.getStateLifetime(n_i,l_i,j_i)
Gamma_ig = 1/T_i #Decay rate of intermediate state

##Rydberg state: 70 S1/2
n_r = 70
l_r = 0
j_r = 0.5
#Account for blackbody stimulation of Rydberg state to nearby states
#Total lifetime is blackbody + radiative: 1/T_rtotal = 1/T_rBB + 1/T_rRadiative
T_rTot = atom.getStateLifetime(
    n_r,l_r,j_r, temperature= 300, includeLevelsUpTo = n_r + 20
    ) 
#Radiative lifetime: decay to other ground state (so at 0 K temp)
T_rRad = atom.getStateLifetime(n_r,l_r,j_r, temperature= 0) 
#Radiative transition from rydberg to intermediate
T_ri = 1/atom.getTransitionRate(n_r,l_r,j_r, n_i,l_i,j_i, temperature=0) 
T_rgp = 1/(1/T_rRad - 1/T_ri) #radiative transition from rydberg to dark ground states

#Black body stimulated: Transition from rydberg to nearby rydberg
T_rBB = 1/(1/T_rTot - 1/T_rRad)

#Set decay rate to each state
Gamma_ri = 1/T_ri
Gamma_rrp = 1/T_rBB # r to r' (r prime)
Gamma_rgp = 1/T_rgp #rydberg to g' (ground prime dark states)

Gamma_rTot = Gamma_ri + Gamma_rrp + Gamma_rgp #Total decay from Rydberg


###Set system parameters:
Rabi_1 = 2*np.pi * 174 * MHz #Blue Rabi freq.
Rabi_2 = 2*np.pi * 115 * MHz #Red Rabi freq.
Delta_1 = 2*np.pi * 1000 * MHz #Detuning for 3 lvl system at the i state

del_total = 2*np.pi * 0 * MHz #2 photon detuning

##Set time grid:
t_0 = 0 #Initial time
t_f = 200 * ns #Final time
nt = 2000 #number of time points
delta_t = (t_f - t_0)/2
tlist = np.linspace(0, t_f, nt)


In [6]:
class Propagator:
    def __init__(self, no_of_steps, delta_t, Rabi_1, Rabi_2):
        self.delta_t = delta_t

        self.x = tf.constant(
            [[0, 1], [1, 0]], dtype=tf.complex128
        )
        self.y = tf.constant(
            [[0 + 0j, 0 - 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
        )

        self.generators =  tf.stack([self.x, self.y])
        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.float64), dtype=tf.float64
        )

        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def infidelity(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x, propagator))
        return 1 - tf.math.real(tr * tf.math.conj(tr)) / (2 ** 2)

propagator = Propagator(1000, 0.001)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 2], -1, 1, dtype=tf.float64)
)

def optimization_step():
    with tf.GradientTape() as tape:
        infidelity = propagator.infidelity()
    gradients = tape.gradient(infidelity, [propagator.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitudes]))
    return infidelity

steps = range(100)
for step in steps:
    current_infidelity = optimization_step()
    print('step %2d: infidelity=%2.5f' %
          (step, current_infidelity))
    
propagator.ctrl_amplitudes.numpy()

step  0: infidelity=0.99994
step  1: infidelity=0.99850
step  2: infidelity=0.99553
step  3: infidelity=0.99077
step  4: infidelity=0.98414
step  5: infidelity=0.97555
step  6: infidelity=0.96494
step  7: infidelity=0.95229
step  8: infidelity=0.93754
step  9: infidelity=0.92068
step 10: infidelity=0.90170
step 11: infidelity=0.88062
step 12: infidelity=0.85748
step 13: infidelity=0.83231
step 14: infidelity=0.80520
step 15: infidelity=0.77623
step 16: infidelity=0.74554
step 17: infidelity=0.71324
step 18: infidelity=0.67951
step 19: infidelity=0.64454
step 20: infidelity=0.60851
step 21: infidelity=0.57166
step 22: infidelity=0.53423
step 23: infidelity=0.49645
step 24: infidelity=0.45861
step 25: infidelity=0.42097
step 26: infidelity=0.38380
step 27: infidelity=0.34738
step 28: infidelity=0.31196
step 29: infidelity=0.27781
step 30: infidelity=0.24515
step 31: infidelity=0.21420
step 32: infidelity=0.18516
step 33: infidelity=0.15817
step 34: infidelity=0.13337
step 35: infidelity=

array([[-1.01993922, -0.55323476],
       [-0.27624327,  0.83300587],
       [-0.59400412, -0.05293211],
       ...,
       [ 0.12166343,  0.39881706],
       [ 0.18901535, -0.67999529],
       [-0.19219344, -0.80598226]])